In [ ]:
username = "akapoor"
import getpass
import os
import uproot
import glob
import pandas as pd
from tqdm import tqdm

if os.system('klist | grep Default | grep ' + username + '@CERN.CH'):
    os.system('echo %s' % getpass.getpass() + " | kinit " + username)

In [ ]:
root_files = '/eos/user/a/akapoor/SWAN_projects/electron_ntuple_TTToSemiLeptonic_TuneCP5_14TeV-_Run3Summer19MiniAOD-2023Scenario.root'

f = uproot.open(root_files)
f.allkeys()

In [1]:
unneccesary_columns = ['ele_index',
       'Spring16GPV1Vals', 'Spring16GPV1RawVals', 'Spring16HZZV1Vals',
       'Spring16HZZV1RawVals', 'Fall17NoIsoV2Vals',
       'Fall17IsoV2Vals', 'Fall17IsoV1Vals',
       'Fall17NoIsoV1Vals', 'Spring16GPV1wp80', 'Spring16GPV1wp90',
       'Spring16HZZV1wpLoose', 'Fall17noIsoV2wp80',
       'Fall17noIsoV2wp90', 'Fall17isoV2wpHZZ', 'Fall17isoV2wp80',
       'Fall17isoV2wp90', 'Fall17noIsoV1wp90',
       'Fall17noIsoV1wp80', 'Fall17noIsoV1wpLoose', 'Fall17isoV1wp90',
       'Fall17isoV1wp80', 'nEvent', 'nRun', 'nLumi']


In [2]:
def get_df(root_file_name):
    f = uproot.open(root_file_name)
    if len(f.allkeys()) == 0:
        return pd.DataFrame()
    df = uproot.open(root_file_name)["ntuplizer/tree"].pandas.df()
    return df.drop(unneccesary_columns, axis=1)

In [ ]:
df = get_df(root_files)
df.columns

import numpy as np

df = df.query("matchedToGenEle != 2")
df.loc[df["matchedToGenEle"] != 1, "matchedToGenEle"] = 0


df = df.query("abs(scl_eta) < 2.5")
df = df.query("ele_pt >= 5")
#ele_fbrem
df['ele_fbrem']=df['ele_fbrem'].astype(float)
df['ele_convDist']=df['ele_convDist'].astype(float)
df['ele_convDcot']=df['ele_convDcot'].astype(float)

df = df.query("ele_fbrem >= (-0.5)")
df = df.query("ele_convDist >= (-2000)")
df = df.query("ele_convDcot >= (-2000)")
df = df.query("ele_kfhits >= 0")
df = df.query("ele_kfchi2 >= 0")

df_barrel = df.query("abs(scl_eta) < 1.48")
df_endcap = df.query("abs(scl_eta) > 1.57")
